# Producing Code from a Project Plans
This notebook is for experimenting with having an LLM write code based on a project plan.

In [ ]:
import os
import subprocess
import warnings
from pathlib import Path

from dotenv import load_dotenv
import openai

warnings.filterwarnings("ignore")
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OPENAI_API_KEY is set correctly.")
else:
    print("OPENAI_API_KEY is not set.")

In [ ]:
code_policies = [
    {
        "name": "Readability and Maintainability (CLEAN Code Principles)",
        "description": "Ensure the code is easy to read, understand, and extend.",
        "policies": [
            "Use meaningful names for functions, variables, and files.",
            "Use correct parts of speach for variable names  (nouns) and functions (verbs).",
            "Keep functions short and focused.",
            "Avoid deep nesting of conditions and loops.",
            "Follow consistent coding style guides (e.g., PEP8 for Python).",
        ],
    },
    {
        "name": "Documentation",
        "description": "Make the code easy to understand for future contributors.",
        "policies": [
            "Add docstrings to functions, classes, and modules.",
            "Include examples of inputs/outputs in docstrings.",
            "Use comments only when the intent is not obvious from the code.",
        ],
    },
    {
        "name": "Configuration Management",
        "description": "Avoid hardcoding settings and paths.",
        "policies": [
            "Use a config file (e.g., config.json, .env) for environment-specific variables.",
            "Keep configurations separate from the core logic.",
            "Validate configuration files during initialization.",
        ],
    },
    {
        "name": "Error Handling and Logging",
        "description": "Ensure graceful error recovery and meaningful logs.",
        "policies": [
            "Handle CSV-related errors explicitly (e.g., parsing, missing columns).",
            "Provide clear error messages for failures.",
            "Implement logging for key events and errors.",
            "Avoid catching generic exceptions (e.g., except Exception).",
        ],
    },
    {
        "name": "Dependency Management",
        "description": "Keep external dependencies minimal and well-documented.",
        "policies": [
            "Use standard libraries where possible.",
            "Ensure dependencies are up-to-date and stable.",
        ],
    },
    {
        "name": "Functions over Classes",
        "description": "Avoid unnecessary functions and state.",
        "policies": [
            "Use module level functions where possible.",
            "Prefer functions over classes for stateless operations.",
            "Prefer stateless operations over stateful operations.",
        ],
    },
    {
        "name": "Avoid Unnecessary Indirection",
        "description": "Reduce redundant layers of function calls to improve readability and maintainability.",
        "policies": [
            "Avoid creating functions that solely wrap another function call without adding meaningful logic.",
            "Directly use the underlying function where possible, unless abstraction is needed for clarity or reuse.",
            "Ensure any intermediary function adds value, such as error handling, parameter validation, or domain-specific logic.",
        ],
    },
]

policy_list = list(
    set(policy for item in code_policies for policy in item.get("policies", []))
)
code_policies = "\n\t- ".join(policy_list)

In [ ]:
code_format_policies = [
    {
        "name": "Avoid non-code content",
        "description": "Ensure that the response is restricted to the Python code.",
        "policies": [
            "Only respond with Python code.",
            "Do not include non-code content.",
            "Any additional content should be code comments or docstrings.",
            "The response should not include any other markdown, just the bare python code.",
        ],
    },
]
policy_list = list(
    set(policy for item in code_format_policies for policy in item.get("policies", []))
)
format_policies = "\n\t- ".join(policy_list)

In [ ]:
code_prompt_preamble = """
You are a software engineer tasked with writing a single python module that has code from a project
plan provided by an architect.
Your goal is to write concise Python that follows best practices in software architecture and coding.
"""

In [ ]:
def format_code_response(response):
    "Removes back-ticks and associated lines (including the language ID )."
    code = ""
    for line in response.split("\n"):
        if not line.startswith("```"):
            code += line + "\n"
    return code

In [ ]:
client = openai.OpenAI(api_key=openai_api_key)

for project_directory in Path("projects").glob("*"):
    project_name = project_directory.name
    # read in the project's "plan.md" file
    plan_file_path = project_directory / "plan.md"
    if not plan_file_path.exists():
        print(f"Skipping project {project_name} as 'plan.md' file is missing.")
        continue
    with open(plan_file_path, "r") as plan_file:
        plan_text = plan_file.read()
    code_prompt = f"""{code_prompt_preamble}
    Your code should adhere to the following policies:\n {code_policies}\n
    Format the response according to these policies:\n {format_policies}\n
    And implement the plan described in this project plan:\n {plan_text}\n
    """
    response = client.chat.completions.create(
        model="o4-mini",
        messages=[{"role": "user", "content": code_prompt}]
    )
    code_response = response.choices[0].message.content
    code = format_code_response(code_response)
    # Create directory and write file
    projects_path = Path("projects")
    projects_path.mkdir(exist_ok=True)
    project_path = projects_path / project_name
    project_path.mkdir(exist_ok=True)
    code_path = project_path / "code.py"
    code_path.write_text(code)

    # format the python code
    subprocess.run(["black", str(code_path)])
    subprocess.run(["isort", str(code_path)])